In [1]:
## Getting to Main directory
import os
os.chdir("../")

In [2]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv



e:\proj\ai assignment\QA_assistant\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
gemini_api_key=os.getenv("GEMINI_API_KEY")
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))



### Data ingestion - Taking pdf documents and Cleaning and Transforming  Data into vector index

In [5]:
from langchain_community.document_loaders import DirectoryLoader

In [6]:
loader = DirectoryLoader('Data', glob="**/*.pdf")

In [7]:
docs = loader.load()

In [8]:
len(docs)

8

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
text_chunks = text_splitter.split_documents(docs)

In [10]:
len(text_chunks)

59

In [11]:
# Clean up our Documents' content
import re

def clean_up_text(content: str) -> str:
    """
    Remove unwanted characters and patterns in text input.

    :param content: Text input.
    
    :return: Cleaned version of original text input.
    """

    # Fix hyphenated words broken by newline
    content = re.sub(r'(\w+)-\n(\w+)', r'\1\2', content)

    # Remove specific unwanted patterns and characters
    unwanted_patterns = [
        "\\n", "  —", "——————————", "—————————", "—————",
        r'\\u[\dA-Fa-f]{4}', r'\uf075', r'\uf0b7'
    ]
    for pattern in unwanted_patterns:
        content = re.sub(pattern, "", content)

    # Fix improperly spaced hyphenated words and normalize whitespace
    content = re.sub(r'(\w)\s*-\s*(\w)', r'\1-\2', content)
    content = re.sub(r'\s+', ' ', content)

    return content

# Call function
cleaned_docs = []
for d in text_chunks: 
    cleaned_text = clean_up_text(d.page_content)
    d.page_content = cleaned_text
    cleaned_docs.append(d)

In [12]:
text_chunks[0].page_content

'111111A Tale of Two BirdsA mother bird and her two young ones lived in a forest. • The mother was killed in a storm and the young birds wereseparated from each other. • Each found a different home.THERE once lived a bird and her two new-born babies in a forest. They had a nest in a tall, shady tree and there the mother bird took care of her little ones day and night.One day, there was a big storm. There was thunder, lightning and rain, and the wind blew down many trees. The tall tree in which the birds lived also came down. A big, heavy branch hit the nest and killed the bird. Fortunately for the baby birds, the strong wind blew them away to the other side of the forest. One of them came down near a cave where a gang of robbers lived. The other landed outside a rishi’s ashram a little distance away.Rationalised 2023-242/A Pact with the Sun'

In [13]:
cleaned_docs[0].metadata

{'source': 'Data\\fepw101.pdf'}

### Configuring Gemini model and GeminiEmbedding

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [15]:
genai.configure(api_key=gemini_api_key)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# Setting Tempreture to 0.3 for getting low risk results

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",api_key=gemini_api_key,temperature=0.3)

### Using FAISS as our vector database taking the index to save them locally

In [18]:
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
vector_store.save_local("faiss_index")

### Applying proper prompts to get best output

In [93]:
prompt_template = """
Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
Context:\n {context}?\n
Question: \n{question}\n

Answer:
"""

# model = ChatGoogleGenerativeAI(model="gemini-pro",
#                             temperature=0.3)

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

prompt
# return chain

PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in\nprovided context just say, "answer is not available in the context", don\'t provide the wrong answer\n\n\nContext:\n {context}?\n\nQuestion: \n{question}\n\n\nAnswer:\n')

In [94]:
from langchain.prompts.chat import ChatPromptTemplate
qa_prompt = ChatPromptTemplate.from_messages([("human", prompt_template)])
qa_prompt


ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in\nprovided context just say, "answer is not available in the context", don\'t provide the wrong answer\n\n\nContext:\n {context}?\n\nQuestion: \n{question}\n\n\nAnswer:\n'))])

In [21]:
chain = load_qa_chain(llm=model, chain_type="stuff", verbose=True, prompt=qa_prompt)


In [22]:
user_question = "What happened in the deadly mongoose story  "

In [23]:
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(user_question)


In [24]:
docs[0]

Document(page_content='Both the baby and the mongoose grew. In five or six months the mongoose had grown to its full size — a lovely animal with two shining black eyes and a bushy tail. The farmer’s son was still a baby in the cradle, sleeping and crying alternately.One day, the farmer’s wife wanted to go to the market. She fed the baby and rocked him to sleep in his little cradle. Picking up the basket, she said to her husband, “I’m off to the bazar. The baby is sleeping. Keep an eye on him. Frankly, I don’t like to leave the child alone with the mongoose.” “You needn’t be afraid,” said the farmer. “The mongoose is a friendly animal. It’s as sweet as our baby and they are the best of friends, you know.”The wife went away, and the farmer, having nothing to do in the house, decided to go out and take a look at his fields not far away. He ran into some friends on the way back and didn’t return for quite some time.The farmer’s wife returned home from the market carrying aheavy basket.', m

In [25]:
response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

e:\proj\ai assignment\QA_assistant\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer


Context:
 Both the baby and the mongoose grew. In five or six months the mongoose had grown to its full size — a lovely animal with two shining black eyes and a bushy tail. The farmer’s son was still a baby in the cradle, sleeping and crying alternately.One day, the farmer’s wife wanted to go to the market. She fed the baby and rocked him to sleep in his little cradle. Picking up the basket, she said to her husband, “I’m off to the bazar. The baby is sleeping. Keep an eye on him. Frankly, I don’t like to leave the child alone with the mongoose.” “You needn’t be afraid,” said the farmer. “The mongoose is a friendly animal. It’s as sweet as our 

In [26]:
print(response["output_text"])

## Deadly Mongoose Story: Incomplete Information

The provided context gives a glimpse into the beginning of the "deadly mongoose" story but doesn't offer the complete narrative or the event where the mongoose displays deadly behavior. 

**Here's what we know from the context:**

* A baby and a mongoose are growing up together in a farmer's household.
* The mongoose is considered friendly and is seen as the baby's friend.
* The farmer's wife expresses concern about leaving the baby alone with the mongoose but is reassured by her husband.
* Both the farmer and his wife leave the house, leaving the baby alone with the mongoose.

**Missing information:**

* We don't know what happens while the baby is alone with the mongoose. 
* The context doesn't reveal the event where the mongoose acts in a deadly manner.
* The story's ending and the fate of the baby and the mongoose remain unknown.

**Therefore, based on the provided context, I cannot fully answer what happened in the "deadly mongoose

In [42]:
eval_questions = [
    "How many baby birds did the mother bird have?",
    "What happened to the mother bird during the storm?",
    "Where did the baby birds end up after the storm?",
    "What happened to one of the baby birds near a cave?",
    "Where did the other baby bird land?",
    "Who came to the forest one day?",
    "What did the king see under a tree near a cave?",
    "What did the king hear from the big, brown bird?",
    "Where did the king go after leaving the cave?",
    "Who welcomed the king at the ashram?",
    "What did the rishi say about the two birds?",
    "What did the king tell the rishi about the birds?",
    "What did the rishi say to the king about the birds?",
    "What did the rishi compare the birds' behavior to?",
    "Which moral best summarizes the story?"
]

eval_answers = [
    "The mother bird had two baby birds.",
    "The mother bird was killed by a heavy branch during the storm.",
    "The baby birds ended up separated from each other after the storm.",
    "One of the baby birds came down near a cave where a gang of robbers lived.",
    "The other baby bird landed outside a rishi’s ashram a little distance away.",
    "The king of the country came to the forest one day.",
    "The king saw a big, brown bird under a tree near a cave.",
    "The king heard the big, brown bird urging someone to take his jewels and horse quickly.",
    "After leaving the cave, the king came to a clearing that looked like an ashram.",
    "The king was welcomed at the ashram by a big, brown bird.",
    "The rishi said that one bird had made friends with robbers while the other welcomed people to the ashram.",
    "The king told the rishi the story of the two birds and their different behaviors.",
    "The rishi said that one is known by the company one keeps.",
    "The rishi compared the birds' behavior to the influence of companionship.",
    "The moral 'One is known by the company one keeps' best summarizes the story."
]


In [44]:
examples = [
    {"query": q, "ground_truth": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [45]:
examples

[{'query': 'How many baby birds did the mother bird have?',
  'ground_truth': ['The mother bird had two baby birds.']},
 {'query': 'What happened to the mother bird during the storm?',
  'ground_truth': ['The mother bird was killed by a heavy branch during the storm.']},
 {'query': 'Where did the baby birds end up after the storm?',
  'ground_truth': ['The baby birds ended up separated from each other after the storm.']},
 {'query': 'What happened to one of the baby birds near a cave?',
  'ground_truth': ['One of the baby birds came down near a cave where a gang of robbers lived.']},
 {'query': 'Where did the other baby bird land?',
  'ground_truth': ['The other baby bird landed outside a rishi’s ashram a little distance away.']},
 {'query': 'Who came to the forest one day?',
  'ground_truth': ['The king of the country came to the forest one day.']},
 {'query': 'What did the king see under a tree near a cave?',
  'ground_truth': ['The king saw a big, brown bird under a tree near a cave

In [46]:
user_question = eval_questions[5]

In [47]:
user_question

'Who came to the forest one day?'

In [48]:
new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(user_question)

docs

[Document(page_content='would the crocodile tell his wife?Rationalised 2023-24', metadata={'source': 'Data\\fepw105.pdf'}),
 Document(page_content='Rationalised 2023-24', metadata={'source': 'Data\\fepw106.pdf'}),
 Document(page_content='Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24', metadata={'source': 'Data\\fepw104.pdf'}),
 Document(page_content='Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood near a cave. Suddenly he heard a voice cry out, “Quick! Hurry up! There’s someone under the tree. Come and take his jewels and his horse. Hurry, or else he’ll slip away.” Th

In [49]:
response = chain(
        {"input_documents":docs, "question": user_question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer


Context:
 would the crocodile tell his wife?Rationalised 2023-24

Rationalised 2023-24

Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24

Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood near a cave. Suddenly he heard a voice cry

In [50]:
response["input_documents"][0].page_content

'would the crocodile tell his wife?Rationalised 2023-24'

In [51]:
print(response["output_text"])

## Who Came to the Forest? 

**The king of the country** came to the forest one day to hunt. 



In [52]:

import nest_asyncio

nest_asyncio.apply()


In [53]:
response

{'input_documents': [Document(page_content='would the crocodile tell his wife?Rationalised 2023-24', metadata={'source': 'Data\\fepw105.pdf'}),
  Document(page_content='Rationalised 2023-24', metadata={'source': 'Data\\fepw106.pdf'}),
  Document(page_content='Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24', metadata={'source': 'Data\\fepw104.pdf'}),
  Document(page_content='Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood near a cave. Suddenly he heard a voice cry out, “Quick! Hurry up! There’s someone under the tree. Come and take his jewels and his horse. Hurry, or el

## RAG model Evaluation using RAGAS; Currently RAGAS is not supporting Gemini, but I do hope it would be available in future. Regardless of that, Integrating Open AI with RAGAS won't be a problem if we already have access to it's api key

In [55]:
from ragas.langchain.evalchain  import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

# create evaluation chains
# faithfulness_chain = RagasEvaluatorChain(metric=faithfulness)
# answer_rel_chain = RagasEvaluatorChain(metric=answer_relevancy)
# context_rel_chain = RagasEvaluatorChain(metric=context_precision)
# context_recall_chain = RagasEvaluatorChain(metric=context_recall)

In [56]:
import pandas as pd

In [57]:
# Create a DataFrame
test_df = pd.DataFrame({
    'question': eval_questions,
    'ground_truth': eval_answers
})

# Print the DataFrame
print(test_df)

                                             question  \
0       How many baby birds did the mother bird have?   
1   What happened to the mother bird during the st...   
2    Where did the baby birds end up after the storm?   
3   What happened to one of the baby birds near a ...   
4                 Where did the other baby bird land?   
5                     Who came to the forest one day?   
6     What did the king see under a tree near a cave?   
7    What did the king hear from the big, brown bird?   
8       Where did the king go after leaving the cave?   
9                Who welcomed the king at the ashram?   
10        What did the rishi say about the two birds?   
11  What did the king tell the rishi about the birds?   
12  What did the rishi say to the king about the b...   
13  What did the rishi compare the birds' behavior...   
14             Which moral best summarizes the story?   

                                         ground_truth  
0                 The mother bi

In [58]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [59]:
test_questions = test_questions[:2]
test_groundtruths = test_groundtruths[:2]

In [60]:
test_questions

['How many baby birds did the mother bird have?',
 'What happened to the mother bird during the storm?']

In [61]:
answers = []
contexts = []

for question in test_questions:
      new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
      docs = new_db.similarity_search(user_question)

      response = chain(
            {"input_documents":docs, "question": question})
      # response = retrieval_augmented_qa_chain.invoke({"question" : question})
      answers.append(response["output_text"])
      contexts.append([context.page_content for context in response["input_documents"]])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer


Context:
 would the crocodile tell his wife?Rationalised 2023-24

Rationalised 2023-24

Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24

Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood near a cave. Suddenly he heard a voice cry

In [62]:
test_groundtruths

['The mother bird had two baby birds.',
 'The mother bird was killed by a heavy branch during the storm.']

In [67]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [80]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truths" : test_groundtruths
})

In [88]:
from datasets import Dataset, Features, Sequence, Value

# Convert each ground truth string into a list containing that string
test_groundtruths_list = [[gt] for gt in test_groundtruths]

# Define the features with the correct type for "ground_truths"
features = Features({
    "question": Value("string"),
    "answer": Value("string"),
    "contexts": Value("string"),
    "ground_truths": Sequence(Sequence(Value("string")))
})

# Create the dataset
response_dataset = Dataset.from_dict({
    "question": test_questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": test_groundtruths_list
})

# Now, you should be able to use the dataset without encountering the ValueError


In [89]:
response_dataset[0]

{'question': 'How many baby birds did the mother bird have?',
 'answer': '## Analyzing the Context for Baby Bird Count:\n\nThe provided text describes a story involving a king, a deer, and birds but **does not specify the number of baby birds the mother bird had**. Therefore, based on the context provided, the answer is: \n\n**Answer is not available in the context** \n',
 'contexts': ['would the crocodile tell his wife?Rationalised 2023-24',
  'Rationalised 2023-24',
  'Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24',
  'Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood

In [90]:
response_dataset[0]

{'question': 'How many baby birds did the mother bird have?',
 'answer': '## Analyzing the Context for Baby Bird Count:\n\nThe provided text describes a story involving a king, a deer, and birds but **does not specify the number of baby birds the mother bird had**. Therefore, based on the context provided, the answer is: \n\n**Answer is not available in the context** \n',
 'contexts': ['would the crocodile tell his wife?Rationalised 2023-24',
  'Rationalised 2023-24',
  'Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24',
  'Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood

In [ ]:

advanced_retrieval_results = evaluate(response_dataset, metrics)